In [ ]:
import add_packages
import json
from toolkit.langchain import (
	models, graphs, tools, prompts
)


# [Quick Start](https://langchain-ai.github.io/langgraph/tutorials/introduction/)

In [ ]:
memory = graphs.MemorySaver()

In [ ]:
tool_tavily_search = tools.TavilySearchResults(max_results=2)

my_tools = [
	tool_tavily_search
]

In [ ]:
class RequestAssistance(graphs.BaseModel):
	"""Escalate the conversation to an expert. Use this if you are unable to assist directly or if the user requires support beyond your permissions.

	To use this function, relay the user's 'request' so the expert can provide the right guidance.
	"""
	request: str

In [ ]:
llm = models.create_llm(provider='openai', version='gpt-3.5-turbo-0125')

llm = llm.bind_tools(my_tools + [RequestAssistance])

In [ ]:
class State(graphs.TypedDict):
	# Messages have the type "list". The `add_messages` function
	# in the annotation defines how this state key should be updated
	# (in this case, it appends messages to the list, rather than overwriting them)
	messages: graphs.Annotated[list, graphs.add_messages]
	ask_human: bool

graph_builder = graphs.StateGraph(State)

In [ ]:
def chatbot(state: State):
	response: prompts.AIMessage = llm.invoke(state["messages"])
	ask_human = False
	
	if response.tool_calls \
   		and response.tool_calls[0]["name"] == RequestAssistance.__name__:
		ask_human = True
	
	return {
		"messages": [response], "ask_human": ask_human
	}

# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)

In [ ]:
graph_builder.add_node("tools", graphs.ToolNode(tools=[tool_tavily_search]))

In [ ]:
def create_tool_response(response: str, ai_msg: prompts.AIMessage):
	return prompts.ToolMessage(
		content=response,
		tool_call_id=ai_msg.tool_calls[0]["id"],
	)

def node_human(state: State):
	new_msgs = []
	
	if not isinstance(state["messages"][-1], prompts.ToolMessage):
		# Typically, the user will have updated the state during the interrupt.
		# If they choose not to, we will include a placeholder ToolMessage to
		# let the LLM continue.
		new_msgs.append(
			create_tool_response("No response from human.", state["messages"][-1])
		)
	
	return {
		"messages": new_msgs,
		"ask_human": False,
	}

graph_builder.add_node("human", node_human)

def select_next_node(
  state: State
) -> graphs.Literal["human", "tools", "__end__"]:
	if state["ask_human"]:
		return "human"
	return graphs.tools_condition(state)

graph_builder.add_conditional_edges(
	"chatbot",
	select_next_node,
	{
		"human": "human",
		"tools": "tools",
		"__end__": "__end__",
	}
)

In [ ]:
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge(graphs.START, "chatbot")
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("human", "chatbot")

graph = graph_builder.compile(
  checkpointer=memory,
	interrupt_before=["human"],
	# interrupt_after=["tools"],
)

In [ ]:
graphs.display_graph(graph)

In [ ]:
config = {"configurable": {"thread_id": "1"}}

# user_input = "Hi there! My name is Will."
# user_input = "Remember my name?"
# user_input = "I need some expert guidance for building this AI agent. Could you request assistance for me?"
# user_input = "I'm learning LangGraph. Could you do some research on it for me?"
user_input = "Ya that's helpful. Maybe I'll build an autonomous agent with it!"

events = graph.stream(
	{"messages": ("user", user_input)}, config, stream_mode="values",
)
for event in events:
	if "messages" in event:
		event["messages"][-1].pretty_print()

In [ ]:
snapshot = graph.get_state(config)
states = graph.get_state_history(config)

In [ ]:
events = graph.stream(
	None, config, stream_mode="values",
)
for event in events:
	event["messages"][-1].pretty_print()

# Chatbots

## [Customer Support](https://langchain-ai.github.io/langgraph/tutorials/customer-support/customer-support/)

## [Prompt Generation from User Requirements](https://langchain-ai.github.io/langgraph/tutorials/chatbots/information-gather-prompting/)

# Multi-Agent Systems

## [Collaboration](https://langchain-ai.github.io/langgraph/tutorials/multi_agent/multi-agent-collaboration/)

Enable two agents to collaborate on a task

## [Supervision](https://langchain-ai.github.io/langgraph/tutorials/multi_agent/agent_supervisor/)

Use an LLM to orchestrate and delegate to individual agents


## [Hierarchical Teams](https://langchain-ai.github.io/langgraph/tutorials/multi_agent/hierarchical_agent_teams/)

Orchestrate nested teams of agents to solve problems

# RAG

## [Adaptive RAG](https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_adaptive_rag/)


## [Agentic RAG](https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_agentic_rag/)


## [Corrective RAG](https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_crag/)


## [Self-RAG](https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_self_rag/)


## [SQL Agent](https://langchain-ai.github.io/langgraph/tutorials/sql-agent/)

# Agent Architectures

## Planning Agents


### [Plan-and-Execute](https://langchain-ai.github.io/langgraph/tutorials/plan-and-execute/plan-and-execute/)

Implement a basic planning and execution agent


### [Reasoning without Observation](https://langchain-ai.github.io/langgraph/tutorials/rewoo/rewoo/)

Reduce re-planning by saving observations as variables


### [LLMCompiler](https://langchain-ai.github.io/langgraph/tutorials/llm-compiler/LLMCompiler/)

Stream and eagerly execute a DAG of tasks from a planner


## Reflection & Critique


### [Basic Reflection](https://langchain-ai.github.io/langgraph/tutorials/reflection/reflection/)

Prompt the agent to reflect on and revise its outputs


### [Reflexion](https://langchain-ai.github.io/langgraph/tutorials/reflexion/reflexion/)

Critique missing and superfluous details to guide next steps


### [Language Agent Tree Search](https://langchain-ai.github.io/langgraph/tutorials/lats/lats/)

Use reflection and rewards to drive a tree search over agents


### [Self-Discover Agent](https://langchain-ai.github.io/langgraph/tutorials/self-discover/self-discover/)

Analyze an agent that learns about its own capabilities

# Evaluation & Analysis

## [Agent-based](https://langchain-ai.github.io/langgraph/tutorials/chatbot-simulation-evaluation/agent-simulation-evaluation/)

Evaluate chatbots via simulated user interactions


## [In LangSmith](https://langchain-ai.github.io/langgraph/tutorials/chatbot-simulation-evaluation/langsmith-agent-simulation-evaluation/)

Evaluate chatbots in LangSmith over a dialog dataset